<a href="https://colab.research.google.com/github/HemantTiwariGitHub/Embedding2Embedding/blob/master/TheTrainerVG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
 try:
    %tensorflow_version 2.x
except:
    pass

In [41]:
import tensorflow as tf
import tensorflow_hub as hub
from absl import logging
import numpy as np
from PIL import Image as PIL_Image
from io import StringIO
import numpy as np
import PIL.Image as Image
import pathlib


print("\u2022 Using TensorFlow Version:", tf.__version__)
print("\u2022 Using TensorFlow Hub Version: ", hub.__version__)
print('\u2022 GPU Device Found.' if tf.test.is_gpu_available() else '\u2022 GPU Device Not Found. Running on CPU')

• Using TensorFlow Version: 2.1.0-rc1
• Using TensorFlow Hub Version:  0.7.0
• GPU Device Not Found. Running on CPU


In [42]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [0]:
MOBILENET_V2_FEATURE_EXTRACTOR_URI = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
PIXEL =224
INPUT_IMAGE_SIZE = (PIXEL, PIXEL)
OUTPUT_FEATURE_VECTOR_SIZE = 1280
feature_extractor = hub.KerasLayer(MOBILENET_V2_FEATURE_EXTRACTOR_URI,
                                   input_shape=INPUT_IMAGE_SIZE + (3,), 
                                   output_shape=[OUTPUT_FEATURE_VECTOR_SIZE],
                                   trainable=False)
MobileNetModel = tf.keras.Sequential([
        feature_extractor
])

In [0]:
def resize_image(image):
    #print("resizing:") 
    resized_img = image.resize(INPUT_IMAGE_SIZE)
    #resized_img.show()
    resized_img_arr = np.array(resized_img)/255.0
    return resized_img_arr

def getImageEmbedding(regionImage, model):
    resizedImage = resize_image(regionImage)
    #plt.imshow(resizedImage)
    #plt.show()
    embedding = model.predict(resizedImage[np.newaxis, ...])
    #print(embedding)
    return embedding

In [0]:
def readImageFromDrive(imgId, regId):
    drivepath = '/content/drive/My Drive/'
    imagepath = drivepath +  'Region_Image/Image_' + str(imgId)+'_Region_' + str(regId)+'.jpg'
    imageName =  str(imgId) + '_' + str(regId)
    imageGet = tf.keras.utils.get_file(imageName, pathlib.Path(imagepath).as_uri())
    imageRead = Image.open(imageGet)
    return imageRead

In [46]:
imgReg = readImageFromDrive(125,2)
imgEmbedding = getImageEmbedding(imgReg, MobileNetModel )
print(imgEmbedding)

[[0.9606538  1.7073144  1.3611007  ... 0.5518554  0.07739219 0.47317785]]


In [0]:
def readTextFromDrive(imgId, regID):

    drivepath = '/content/drive/My Drive/'
    IembeddingPath = drivepath+'/Region_Descp'
    descpath =  drivepath + 'Region_Descp/Image_' + str(imgId)+'_Region_Descp' + str(regID)+'.txt'
    f = open(descpath, 'r')
    content = f.read()
    return content

In [0]:
def readEmbeddingFromDrive(imgId, regID):

    drivepath = '/content/drive/My Drive/'
    IembeddingPath = drivepath+'/Region_Descp'
    descpath =  drivepath + 'Image_Embedding/Image_' + str(imgId)+'_Embedding_'+ str(regID)+'.txt'
    f = open(descpath, 'r')
    content = f.read()
    return content


In [0]:
#!unzip <filename>

In [0]:
def getFileList(path):
    import glob,os
    iwd = os.getcwd()
    os.chdir(path)
    filelist= []
    for file in glob.glob("*.txt"):
        filelist.append(file)
    os.chdir(iwd)
    return filelist

In [0]:
import re
def getImageIDregID(str1):
    idList = list(map(int, re.findall('\d+',str1)))
    return idList

In [0]:
def getAllPairs(loadpath):
  ListOfFiles = getFileList(loadpath)
  ListOfID = []
  for filename in ListOfFiles:
    #print(filename)
    ID,REGID = getImageIDregID(filename)
    ListOfID.append([ID,REGID])
  return ListOfID

In [53]:
LOADING_PATH = "/content/Region_Descp"
ListOfIDs = getAllPairs(LOADING_PATH)

print(len(ListOfIDs))

56775


In [0]:
def LoadBatch(baseIndex, batchSize, sourceList):
    maxIndex = baseIndex + batchSize
    xList = []
    yList = []
    while (baseIndex < maxIndex):
        if (baseIndex%10==0):
          print("\n\nreading index" + str(baseIndex))
        description = readTextFromDrive(sourceList[baseIndex][0], sourceList[baseIndex][1])
       # print("descp:" + description)
        x = embed([description])

        print("Text Embedding" + str(x))
        

        imgReg = readImageFromDrive(sourceList[baseIndex][0], sourceList[baseIndex][1])
       # display(imgReg)
        y = getImageEmbedding(imgReg, MobileNetModel )
        print("Image Embedding" + str(y))
        xList.append(x)
        yList.append(y)
        baseIndex = baseIndex + 1

    return xList,yList

In [55]:
import numpy as np
np.set_printoptions(threshold=10)
X,Y = LoadBatch(0, 1000, ListOfIDs)

49152/46438 [===============================] - 0s 6us/step


reading index550
8192/901 [================================================================================================================================================================================================================================================================================] - 0s 22us/step


reading index560
8192/5604 [===========================================] - 0s 33us/step


reading index570
16384/11347 [===========================================] - 0s 20us/step


reading index580
40960/33837 [====================================] - 0s 5us/step


reading index590
188416/183725 [==============================] - 0s 1us/step


reading index600
8192/4234 [==========================================================] - 0s 28us/step


reading index610
8192/4076 [============================================================] - 0s 28us/step


reading index620
8192/2262 [==================================

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

tf.keras.backend.clear_session() 

In [145]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(512,), name='textEmbedding')
layer2 = layers.Dense(1024,activation='relu', name='dense_1')(inputs)
layer3 = layers.Dense(1024, activation='relu', name='dense_2')(layer2)
outputs = layers.Dense(1280, activation='relu', name='predictions')(layer3)
output_reshape  = keras.layers.Reshape((1280,))(outputs)

model = keras.Model(inputs=inputs, outputs=output_reshape)


model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
textEmbedding (InputLayer)   [(None, 512)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
predictions (Dense)          (None, 1280)              1312000   
_________________________________________________________________
reshape_3 (Reshape)          (None, 1280)              0         
Total params: 2,886,912
Trainable params: 2,886,912
Non-trainable params: 0
_________________________________________________________________


In [140]:
validationSetSize =int(len(X)/10)
print("ValidationSet Size : " + str(validationSetSize))
x_val = X[-validationSetSize:]
y_val = Y[-validationSetSize:]
x_train = X[:-validationSetSize]
y_train = Y[:-validationSetSize]

y_train - tf.convert_to_tensor(y_train)
#x_train = np.asarray(x_train,dtype="float32", order=None)
#x_train = tf.convert_to_tensor(x_train)
print(x_train[0])
print()
print(y_train[0])




ValidationSet Size : 100
tf.Tensor(
[[ 0.03020312  0.0755768  -0.03746407 ...  0.03053355  0.01503169
  -0.00533399]], shape=(1, 512), dtype=float32)

[[0.5235753  0.15164247 0.915035   ... 0.20287666 0.         1.7772793 ]]


In [0]:
model.compile(optimizer=keras.optimizers.RMSprop(),  # Optimizer
              # Loss function to minimize
              loss=keras.losses.CosineSimilarity(),
              # List of metrics to monitor
              metrics=[keras.metrics.CosineSimilarity()])


In [148]:
print('# Fit model on training data')
history = model.fit(x_train, y_train[0],
                    batch_size=64,
                    epochs=100,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                   )

# Fit model on training data
Train on 1 samples
Epoch 1/100
1/1 [==============================] - 0s 477ms/sample - loss: -0.3779 - cosine_similarity: 0.3779
Epoch 2/100
1/1 [==============================] - 0s 31ms/sample - loss: -0.6535 - cosine_similarity: 0.6535
Epoch 3/100
1/1 [==============================] - 0s 35ms/sample - loss: -0.7399 - cosine_similarity: 0.7399
Epoch 4/100
1/1 [==============================] - 0s 26ms/sample - loss: -0.7796 - cosine_similarity: 0.7796
Epoch 5/100
1/1 [==============================] - 0s 26ms/sample - loss: -0.7981 - cosine_similarity: 0.7981
Epoch 6/100
1/1 [==============================] - 0s 26ms/sample - loss: -0.8034 - cosine_similarity: 0.8034
Epoch 7/100
1/1 [==============================] - 0s 26ms/sample - loss: -0.8064 - cosine_similarity: 0.8064
Epoch 8/100
1/1 [==============================] - 0s 30ms/sample - loss: -0.8074 - cosine_similarity: 0.8074
Epoch 9/100
1/1 [==============================] - 0s 26ms/sample - los